# rosalie dev

Notebook purpose:

- Develop `rosalie` package and templates for working with it

In [1]:
import logging

import numpy as np
import pandas as pd
import statsmodels.api as sm

from causaljet.experiment_evaluation.models import Cuped
import rosalie as ro

# Silence info logging of root logger to silence Cuped logging
logging.getLogger().setLevel(logging.WARNING)

# pd.set_option('display.max_rows', 500)

%config InlineBackend.figure_format ='retina'
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [3]:
# Load data

UNIT_LEVEL = "customer"
UNIT_ID = 'id'
METRICS = ['order_price', 'gmv']
PRE_PERIOD = '1 Jan 2023', '31 May 2023'
POST_PERIOD ='1 Jun 2023', ' 31 Aug 2023'
FORMAT = 'cross_section'

CACHE_PATH = f'/Users/fabian.gunzinger/tmp/rosalie/{UNIT_LEVEL}.csv'

df = (
    ro.DataReader().load_data(
        UNIT_LEVEL,
        cache_path=CACHE_PATH
        )
    .pipe(ro.add_artificial_gmw)
    .pipe(ro.create_pre_post_data, 
          id_col=UNIT_ID,
          metrics=METRICS,
          pre_period=PRE_PERIOD,
          post_period=POST_PERIOD)
)
ro.data_info(df)

Querying data from BigQuery...


In [7]:
def causal_jet_cuped(df, metric):
    """Run Causal Jet CUPED implementation and return p-value.

    Because data is already pre-processed, we only need to supply the following:
    - A cross-section dataframe with `metric` and `metric_pre` columns to `ass_w_cov_panel_df`
    - The metric name to `metric_name`
    - The unit identifier to `unit_identifier`

    All other parameters can be left as default.
    """
    result = Cuped(
        ass_w_cov_panel_df=df,
        metric_name=metric,
        unit_identifier=UNIT_ID,
        cluster_identifier=UNIT_ID,
        is_treated_col='is_treated',
        weight_col='assignments_freq',
        additional_regressors=[],
        start_date=None,
        date_identifier=None,
        lookback=None,
    )._get_results()
    return result.pvalues[1]


def traditional_cuped(df, metric):
    """Run traditional CUPED and return p-value."""
    
    def _cuped_adjusted_metric(df, metric, metric_pre):
        y = df[metric].values
        x = df[metric_pre].values
        valid_indices = (~np.isnan(y)) & (~np.isnan(x))
        y_valid, x_valid = y[valid_indices], x[valid_indices]
        m = np.cov(y_valid, x_valid)
        theta = m[0, 1] / m[1, 1]
        return (y - (x - np.nanmean(x)) * theta)

    # Perform experiment evaluation and return p-value
    # (Use WLS to be consistent with CausalJet)
    y = _cuped_adjusted_metric(df, metric, f"{metric}_pre")
    x = sm.add_constant(df["is_treated"].astype(float))
    w = df["assignments_freq"]
    model = sm.WLS(endog=y, exog=x, weights=w)
    results = model.fit()
    return results.pvalues["is_treated"]

In [10]:
evaluators = [traditional_cuped, causal_jet_cuped]

eval = ro.Simulator(
    df=df,
    metrics=METRICS,
    id_col=UNIT_ID,
    # evaluators=evaluators,
    # sample_min=1000,
    # sample_max=21_000,
    num_steps=10,
    mdes=[0.01, 0.03],
    num_runs=5,
    baseline_evaluator="welch",
    # preprocessors=preprocessors,
    # sample_timestamps=False,
    # alpha=0.05,
    # random_seed=2312,
    # testing=True
    )

result = eval.run()
print(result.data.head())
result.plot()

INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
INFO - Initializing Simulator with specified evaluators: ['welch_t_test']
INFO - Generating datasets...
INFO - Generating datasets...
100%|██████████| 10/10 [00:01<00:00,  7.55it/s]
INFO - Evaluating experiments...
INFO - Evaluating experiments...
100%|██████████| 200/200 [00:00<00:00, 289.73it/s]

  metric     evaluator  sample_size  mdes  power
0    gmv  welch_t_test          100  0.01    0.0
1    gmv  welch_t_test          100  0.03    0.2
2    gmv  welch_t_test         7009  0.01    1.0
3    gmv  welch_t_test         7009  0.03    1.0
4    gmv  welch_t_test        13918  0.01    1.0


alt.VConcatChart(...)